In [ ]:
'''
Organizing code for Program Evaluation homework.
'''

In [1]:
import pandas as pd
import numpy as np
from math import sqrt
from tabulate import tabulate
import statsmodels.api as sm
from patsy import dmatrices
import matplotlib.pyplot as plt
import pprint

%matplotlib inline
pp = pprint.PrettyPrinter(indent=2)

## Read & write data

In [16]:
def read_data(filename):
    '''
    Reads csv, dta, excel file. Returns a DataFrame.
    '''
    assert filename[-3:] in ['csv', 'dta', 'xlsx'], "can only read filename with .csv, .dta, .xlsx extensions"
    if filename[-3:] == 'csv':
        df = pd.read_csv(filename)
    elif filename[-3:] == 'dta':
        df = pd.read_stata(filename)
    elif filename[-4:]=='xlsx':
        df = pd.read_excel(filename)
    return df


def write_data(df, filename):
    '''
    Write DataFrame to file.
    '''
    assert filename[-3:] in ['csv', 'dta', 'xlsx'], "can only write to .csv, .dta"
    if filename[-3:] == 'csv':
        df.to_csv(filename)
    elif filename[-3:] == 'dta':
        df.to_stata(filename)

## Create dummy variables.

In [17]:
'''
Create binary dummy variables from categoricals. using rdtyper as example.
'''
rdtyper_dummy = pd.get_dummies(neg5_pos3.rdtyper)
neg5_pos3 = pd.concat([neg5_pos3, rdtyper_dummy], axis=1)
'''
Create binary dummy from continuous variable, using a threshold.
'''
neg5_pos3['after'] = neg5_pos3.delta_time >= 0
neg5_pos3['after'] = neg5_pos3.after.astype(int)
'''
Create interaction dummies.
'''
neg5_pos3["I_after"] = neg5_pos3.after * neg5_pos3.I
neg5_pos3["A_after"] = neg5_pos3.after * neg5_pos3.A
neg5_pos3["N_after"] = neg5_pos3.after * neg5_pos3.N

AssertionError: can only read filename with .csv, .dta, .xlsx extensions

In [ ]:
def calc_se(std1, std2, n1, n2):
    '''
    Calculate standard error using two std. and sample size.
    
    Input:
        std1: (float)
        std2: (float)
        n1: (int)
        n2: (int)
    '''
    inner = (std1**2 / n1) + (std2**2 / n2)
    sem = sqrt(inner)
    return sem

In [ ]:
'''
run regression
'''
y, X = dmatrices('vcrm ~ A + after + A_after', data=neg5_pos3, return_type='dataframe')
mod = sm.OLS(y, X)
res = mod.fit()
res.summary()
'''
run regresssion with clustered standard error.
'''
# (-5, +3) Target area baseline
y, X = dmatrices('vcrm ~ I + after + I_after', data=neg5_pos3, return_type='dataframe')
mod = sm.OLS(y, X)
cluster_prd_ols = mod.fit(cov_type='cluster', cov_kwds={'groups': neg5_pos3['prd']}, use_t=True)
cluster_prd_ols.summary()